In [1]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time
import nltk

# load the dataset
dataset = pd.read_csv('./data/test.csv')
dataset.head()

,Id,Page content
0,27643,"<html><head><div class=""article-info""><span cl..."
1,27644,"<html><head><div class=""article-info""><span cl..."
2,27645,"<html><head><div class=""article-info""><span cl..."
3,27646,"<html><head><div class=""article-info""><span cl..."
4,27647,"<html><head><div class=""article-info""><span cl..."


# 1. Time: 
### is_weekend, is_mon, is_sat, is_sun

In [2]:
# get weekday
def get_weekday(text: str) -> int: #input html format
    soup = BeautifulSoup(text)
    selector = "time"
    date = [i.text for i in soup.select(selector)][0]
    #print(soup.select(selector))
    #print(date)

    try:
        date = date.split()
        date = str(date[0] + ' ' + date[1])
    #print(date)

        d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        d = d.weekday()+1
    #print(d)
    except:
        d = 1
        
    return d

res = []
for i in range(dataset.shape[0]):
    res.append(get_weekday(dataset['Page content'][i]))
    
dataset['weekday'] = res

In [3]:
# is_weekend
res = []
for i in range(dataset.shape[0]):
    if dataset['weekday'][i] == 6 or dataset['weekday'][i] == 7:
        res.append(1)
    else:
        res.append(0)
    
dataset['is_weekend'] = res

In [4]:
# is_sat
res = []
for i in range(dataset.shape[0]):
    if dataset['weekday'][i] == 6:
        res.append(1)
    else:
        res.append(0)
    
dataset['is_sat'] = res

In [5]:
# is_sun
res = []
for i in range(dataset.shape[0]):
    if dataset['weekday'][i] == 7:
        res.append(1)
    else:
        res.append(0)
    
dataset['is_sun'] = res

In [6]:
# is_mon
res = []
for i in range(dataset.shape[0]):
    if dataset['weekday'][i] == 1:
        res.append(1)
    else:
        res.append(0)
    
dataset['is_mon'] = res

In [7]:
dataset.head(20)

,Id,Page content,weekday,is_weekend,is_sat,is_sun,is_mon
0,27643,"<html><head><div class=""article-info""><span cl...",1,0,0,0,1
1,27644,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0
2,27645,"<html><head><div class=""article-info""><span cl...",2,0,0,0,0
3,27646,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0
4,27647,"<html><head><div class=""article-info""><span cl...",5,0,0,0,0
5,27648,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0
6,27649,"<html><head><div class=""article-info""><span cl...",7,1,0,1,0
7,27650,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0
8,27651,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0
9,27652,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0


# Length
### # of char(title+article), words(title+article), sentence(article)

In [8]:
def get_article(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    article = soup.find('section',{'class':'article-content'})
    return article.get_text()

res = []

for i in range(dataset.shape[0]):
    res.append(get_article(dataset['Page content'][i]))
dataset['article'] = res

In [9]:
def get_title(text: str) -> str:    
    soup = BeautifulSoup(text, "html.parser")
    title = soup.find('h1',{'class':'title'})
    return title.get_text()

res = []

for i in range(dataset.shape[0]):
    res.append(get_title(dataset['Page content'][i]))
dataset['title'] = res

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
#new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"] 
#stop_words = stop_words.union(new_words)

corpus = []
for i in range(0, dataset.shape[0]):
    text = re.sub('[^a-zA-Z]', ' ', dataset['article'][i])
    text = text.lower()
    
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
    
dataset['article_clean'] = corpus

In [11]:
corpus = []
for i in range(0, dataset.shape[0]):
    text = re.sub('[^a-zA-Z]', ' ', dataset['title'][i])
    text = text.lower()
    
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
    
dataset['title_clean'] = corpus

In [12]:
dataset.head(10)

,Id,Page content,weekday,is_weekend,is_sat,is_sun,is_mon,article,title,article_clean,title_clean
0,27643,"<html><head><div class=""article-info""><span cl...",1,0,0,0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...
1,27644,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store
2,27645,"<html><head><div class=""article-info""><span cl...",2,0,0,0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon
3,27646,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee
4,27647,"<html><head><div class=""article-info""><span cl...",5,0,0,0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...
5,27648,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0,"SAG nominations are here, and as usual, the s...",9 outstanding TV performances SAG should have ...,sag nomination usual swatch nominee quite cove...,outstanding tv performance sag honored
6,27649,"<html><head><div class=""article-info""><span cl...",7,1,0,1,0,Florida State quarterback Jameis Winston ...,'Red Lightning' Is a Ball Boy With His Own Hig...,florida state quarterback jameis winston burst...,red lightning ball boy highlight reel
7,27650,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0,Everyone has seen inspirational quotes pop up...,The Truth About Inspirational Quotes,everyone seen inspirational quote pop facebook...,truth inspirational quote
8,27651,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0,You can add it to a motion detector. You can...,Add Wi-Fi to Practically Anything With This Ar...,add motion detector add security camera heck c...,add wi fi practically anything arduino compati...
9,27652,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0,The intimate looks at locations around the gl...,Google Street View Says Japan Is Top Virtual T...,intimate look location around globe made possi...,google street view say japan top virtual touri...


In [13]:
# num_char

res = []

for i in range(dataset.shape[0]):
    res.append(len(dataset['article_clean'][i].replace(" ", "")) + len(dataset['title_clean'][i].replace(" ", "")))
dataset['num_char'] = res

In [14]:
# num_words

res = []

for i in range(dataset.shape[0]):
    res.append(len(dataset['article_clean'][i].split(" ")) + len(dataset['title_clean'][i].split(" ")))
dataset['num_words'] = res

In [15]:
# num_sentence

res = []

for i in range(dataset.shape[0]):
    res.append(len(dataset['article'][i].split(".")))
dataset['num_sentence'] = res

In [16]:
dataset.head(10)

,Id,Page content,weekday,is_weekend,is_sat,is_sun,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence
0,27643,"<html><head><div class=""article-info""><span cl...",1,0,0,0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...,2294,382,37
1,27644,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store,489,84,6
2,27645,"<html><head><div class=""article-info""><span cl...",2,0,0,0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon,620,105,13
3,27646,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee,670,113,7
4,27647,"<html><head><div class=""article-info""><span cl...",5,0,0,0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...,835,146,10
5,27648,"<html><head><div class=""article-info""><span cl...",3,0,0,0,0,"SAG nominations are here, and as usual, the s...",9 outstanding TV performances SAG should have ...,sag nomination usual swatch nominee quite cove...,outstanding tv performance sag honored,1246,210,24
6,27649,"<html><head><div class=""article-info""><span cl...",7,1,0,1,0,Florida State quarterback Jameis Winston ...,'Red Lightning' Is a Ball Boy With His Own Hig...,florida state quarterback jameis winston burst...,red lightning ball boy highlight reel,1474,260,39
7,27650,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0,Everyone has seen inspirational quotes pop up...,The Truth About Inspirational Quotes,everyone seen inspirational quote pop facebook...,truth inspirational quote,331,49,6
8,27651,"<html><head><div class=""article-info""><span cl...",6,1,1,0,0,You can add it to a motion detector. You can...,Add Wi-Fi to Practically Anything With This Ar...,add motion detector add security camera heck c...,add wi fi practically anything arduino compati...,1128,194,17
9,27652,"<html><head><div class=""article-info""><span cl...",4,0,0,0,0,The intimate looks at locations around the gl...,Google Street View Says Japan Is Top Virtual T...,intimate look location around globe made possi...,google street view say japan top virtual touri...,2062,347,22


# NLP
### num_verb, num_adverb, num_noun, title_freq, nonstopword_rate, unique_rate

In [17]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# num_verb
tags = set(['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])

# res = []
# for i in range(dataset.shape[0]):
#     ret = []
#     words = nltk.word_tokenize(dataset['article'][i] + dataset['title'][i])
#     pos_tags = nltk.pos_tag(words)
#     for word, pos in pos_tags:
#             if (pos in tags):
#                 ret.append(word)
#     res.append(ret)
# dataset['num_verb'] = res

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yuwei\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [18]:
# num_noun
tags = set(['NN', 'NNS', 'NNP', 'NNPS', 'PRP', 'PRP$'])

# res = []
# for i in range(dataset.shape[0]):
#     print("step: " + str(i))
#     ret = []
#     words = nltk.word_tokenize(dataset['article'][i] + dataset['title'][i])
#     pos_tags = nltk.pos_tag(words)
#     for word, pos in pos_tags:
#             if (pos in tags):
#                 ret.append(word)
#     res.append(ret)
# dataset['num_noun'] = res

In [19]:
# num_adverb
tags = set(['RB', 'RBR', 'RBS', 'RP'])

# res = []
# for i in range(dataset.shape[0]):
#     print("step: " + str(i))
#     ret = []
#     words = nltk.word_tokenize(dataset['article'][i] + dataset['title'][i])
#     pos_tags = nltk.pos_tag(words)
#     for word, pos in pos_tags:
#             if (pos in tags):
#                 ret.append(word)
#     res.append(ret)
# dataset['num_verb'] = res

In [20]:
# unique_rate

res = []
for i in range(dataset.shape[0]):
    unique_rate = len(set(dataset['article_clean'][i].split(' '))) / len(dataset['article_clean'][i].split(' '))
    res.append(unique_rate)
    
dataset['unique_rate'] = res

In [21]:
# nonstopword_rate

res = []
for i in range(dataset.shape[0]):
    unique_rate = len((dataset['article_clean'][i].split(' '))) / len(dataset['article'][i].split(' '))
    res.append(unique_rate)
    
dataset['nonstopword_rate'] = res

In [22]:
# title_freq

res = []
for i in range(dataset.shape[0]):
    time = 0
    for j in dataset['title_clean'][i].lower().split(" "):
        content = dataset['article_clean'][i].lower().split(" ")
        for k in content:
            if j == k:
                time = time+1
        
    res.append(time/dataset['num_words'][i])
    
dataset['title_freq'] = res

In [23]:
dataset = dataset.drop(columns=['weekday', 'Id'])
dataset.head(10)

,Page content,is_weekend,is_sat,is_sun,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq
0,"<html><head><div class=""article-info""><span cl...",0,0,0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...,2294,382,37,0.704301,0.590476,0.075916
1,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store,489,84,6,0.696203,0.530201,0.178571
2,"<html><head><div class=""article-info""><span cl...",0,0,0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon,620,105,13,0.808081,0.589286,0.142857
3,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee,670,113,7,0.787037,0.666667,0.097345
4,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...,835,146,10,0.839416,0.608889,0.102740
5,"<html><head><div class=""article-info""><span cl...",0,0,0,0,"SAG nominations are here, and as usual, the s...",9 outstanding TV performances SAG should have ...,sag nomination usual swatch nominee quite cove...,outstanding tv performance sag honored,1246,210,24,0.829268,0.498783,0.042857
6,"<html><head><div class=""article-info""><span cl...",1,0,1,0,Florida State quarterback Jameis Winston ...,'Red Lightning' Is a Ball Boy With His Own Hig...,florida state quarterback jameis winston burst...,red lightning ball boy highlight reel,1474,260,39,0.696850,0.659740,0.057692
7,"<html><head><div class=""article-info""><span cl...",1,1,0,0,Everyone has seen inspirational quotes pop up...,The Truth About Inspirational Quotes,everyone seen inspirational quote pop facebook...,truth inspirational quote,331,49,6,0.847826,0.575000,0.102041
8,"<html><head><div class=""article-info""><span cl...",1,1,0,0,You can add it to a motion detector. You can...,Add Wi-Fi to Practically Anything With This Ar...,add motion detector add security camera heck c...,add wi fi practically anything arduino compati...,1128,194,17,0.741935,0.590476,0.082474
9,"<html><head><div class=""article-info""><span cl...",0,0,0,0,The intimate looks at locations around the gl...,Google Street View Says Japan Is Top Virtual T...,intimate look location around globe made possi...,google street view say japan top virtual touri...,2062,347,22,0.646884,0.598579,0.141210


# Link

In [24]:
def find_link(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    link = soup.find_all('a')
    
    return len(link)

def find_img(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    img = soup.find_all('img')
    
    return len(img)

def find_video(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    video = soup.find_all('video')
    
    return len(video)

def find_audio(content: str)-> int:
    soup = BeautifulSoup(content, 'html.parser')
    audio = soup.find_all('audio')
    
    return len(audio)

def find_total(content: str)-> int:
    return find_link(content) + find_img(content) + find_video(content) + find_audio(content)

# # for test
# def find_script(content: str)-> int:
#     soup = BeautifulSoup(content, 'html.parser')
#     script = soup.find_all('script')
#     #print(script)
    
#     return len(script)

img_list = []
link_list = []
video_list = []
audio_list = []
total_list = []

for n  in range(dataset.shape[0]):
    img_list.append(find_img(dataset.loc[n,'Page content']))
    link_list.append(find_link(dataset.loc[n,'Page content']))
    video_list.append(find_video(dataset.loc[n,'Page content']))
    audio_list.append(find_audio(dataset.loc[n,'Page content']))
    total_list.append(find_total(dataset.loc[n,'Page content']))
        
dataset['# of img'] = img_list
dataset['# of link'] = link_list
dataset['# of video'] = video_list
dataset['# of audio'] = audio_list
dataset['# of total'] = total_list

In [25]:
dataset.head(10)

,Page content,is_weekend,is_sat,is_sun,is_mon,article,title,article_clean,title_clean,num_char,num_words,num_sentence,unique_rate,nonstopword_rate,title_freq,# of img,# of link,# of video,# of audio,# of total
0,"<html><head><div class=""article-info""><span cl...",0,0,0,1,Note to humanity: One Direction fandom ai...,Soccer Star Gets Twitter Death Threats After T...,note humanity one direction fandom nothin mess...,soccer star get twitter death threat tackling ...,2294,382,37,0.704301,0.590476,0.075916,1,30,0,0,31
1,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Shortly after announcing a hardware upgrade f...,Google Glass Gets an Accessory Store,shortly announcing hardware upgrade google gla...,google glass get accessory store,489,84,6,0.696203,0.530201,0.178571,3,13,0,0,16
2,"<html><head><div class=""article-info""><span cl...",0,0,0,0,"Well, that was quick. Just hours after going ...",OUYA Gaming Console Already Sold Out on Amazon,well quick hour going sale u canada uk ouya ga...,ouya gaming console already sold amazon,620,105,13,0.808081,0.589286,0.142857,2,13,0,0,15
3,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Between Two Ferns: Oscar Buzz Edition Part 1...,'Between Two Ferns' Mocks Oscar Nominees,two fern oscar buzz edition part zach galifian...,two fern mock oscar nominee,670,113,7,0.787037,0.666667,0.097345,1,15,0,0,16
4,"<html><head><div class=""article-info""><span cl...",0,0,0,0,Ever since The Hurt Locker it seems like ...,'American Sniper' Trailer: Looks Like Eastwood...,ever since hurt locker seems like military the...,american sniper trailer look like eastwood may...,835,146,10,0.839416,0.608889,0.102740,1,10,0,0,11
5,"<html><head><div class=""article-info""><span cl...",0,0,0,0,"SAG nominations are here, and as usual, the s...",9 outstanding TV performances SAG should have ...,sag nomination usual swatch nominee quite cove...,outstanding tv performance sag honored,1246,210,24,0.829268,0.498783,0.042857,2,6,0,0,8
6,"<html><head><div class=""article-info""><span cl...",1,0,1,0,Florida State quarterback Jameis Winston ...,'Red Lightning' Is a Ball Boy With His Own Hig...,florida state quarterback jameis winston burst...,red lightning ball boy highlight reel,1474,260,39,0.696850,0.659740,0.057692,1,22,0,0,23
7,"<html><head><div class=""article-info""><span cl...",1,1,0,0,Everyone has seen inspirational quotes pop up...,The Truth About Inspirational Quotes,everyone seen inspirational quote pop facebook...,truth inspirational quote,331,49,6,0.847826,0.575000,0.102041,1,10,0,0,11
8,"<html><head><div class=""article-info""><span cl...",1,1,0,0,You can add it to a motion detector. You can...,Add Wi-Fi to Practically Anything With This Ar...,add motion detector add security camera heck c...,add wi fi practically anything arduino compati...,1128,194,17,0.741935,0.590476,0.082474,2,17,0,0,19
9,"<html><head><div class=""article-info""><span cl...",0,0,0,0,The intimate looks at locations around the gl...,Google Street View Says Japan Is Top Virtual T...,intimate look location around globe made possi...,google street view say japan top virtual touri...,2062,347,22,0.646884,0.598579,0.141210,3,23,0,0,26


In [26]:
dataset.to_csv('./data/test_2.csv')